## Network Improvements
---
Use this section to add new links or improve existing ones. New links should be drawn in a GIS program. Functions for adding reference columns and splitting existing links will be added in the future, but need to be manually added for now. If wanting to see the impacts of only one improvement/new link, be sure to export that as a separate network.

In [ ]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np

import networkx as nx
from tqdm import tqdm
import pickle

import src.conflation_tools as conflation_tools
import src.add_attributes as add_attributes

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

import bicycle_facilities.matching_script as matching_script

In [ ]:
links = gpd.read_file(config['network_fp'] / 'networks.gpkg',layer='osm_links')

# filter to roads
links = links[links['link_type']=='road']

#add street names back to osm
#add attributes back (especially the oneway column)
osm_attrs = gpd.read_file(config['osmdwnld_fp'] / f"osm_{config['geofabrik_year']}.gpkg",layer='raw',ignore_geometry=True)
cols_to_keep = ['osmid','highway','name']
links = pd.merge(links,osm_attrs[cols_to_keep],on='osmid')
del osm_attrs

#format name attrbute
links['name2'] = links['name'].apply(lambda row: conflation_tools.remove_suffix(row))

In [ ]:
# #TODO re-order this import routing network  
# # import test case/ base case (think of a better word) network
# with (config['calibration_fp']/"calibration_network.pkl").open('rb') as fh:
#     links,turns = pickle.load(fh)
# del turns
# links['facility_rev'] = None

# #format name attrbute
# links['name2'] = links['name'].apply(lambda row: conflation_tools.remove_suffix(row))

# GDOT Study Area

In [ ]:
buffer_ft = 100
max_hausdorff_dist = 500
other_name = 'coa'

In [ ]:
# import improvements
improvements = gpd.read_file(config['bicycle_facilities_fp']/"network_modifications.gpkg",layer=f'{other_name}')[[f'{other_name}_id',f'{other_name}_name',f'{other_name}_osm_type','geometry']]

In [ ]:
# copy to prevent modification
links_buffered = links.copy()
other_source = improvements.copy()

# buffer the osm cycleways
links_buffered.geometry = links_buffered.buffer(buffer_ft)

# intersect with coa/arc (returns coa/arc linestrings)
overlap = gpd.overlay(other_source,links_buffered)

#street name check if for bike lanes / sharrows / cycletracks
overlap['name'] = overlap['name'].apply(lambda row: matching_script.remove_suffix(row))
overlap[f"{other_name}_name"] = overlap[f"{other_name}_name"].apply(lambda row: matching_script.remove_suffix(row))
overlap['name_check'] = overlap.apply(lambda row: matching_script.name_check(row['name'],row[f"{other_name}_name"]),axis=1)

In [ ]:
# only accept matching names
overlap = overlap[(overlap['name_check']==True)]

In [ ]:
overlap.drop_duplicates(inplace=True)

In [ ]:
### HAUSDORFF DISTANCE CHECK ###
# add osm geometry to compare against arc/coa geometry
overlap = pd.merge(overlap,links[['osm_linkid','geometry']],on='osm_linkid')
overlap['hausdorff_dist'] = overlap.apply(lambda row: row['geometry_x'].hausdorff_distance(row['geometry_y']),axis=1)
overlap.drop(columns=['geometry_x'],inplace=True)
overlap.rename(columns={'geometry_y':'geometry'},inplace=True)

# replace intersected geometry with the original geometry
overlap = gpd.GeoDataFrame(overlap,geometry='geometry')

In [ ]:
# for remaining multi matches choose match with the lowest hausdorff distance
min_hausdorff = overlap.groupby('osm_linkid')['hausdorff_dist'].idxmin()
overlap = overlap.loc[min_hausdorff]

In [ ]:
overlap.explore()

## Visually, it looks like most of the improvements were properly assigned

In [ ]:
improvements_buffer = improvements.copy()
improvements_buffer.geometry = improvements_buffer.buffer(200)
m = improvements_buffer.explore(f'{other_name}_osm_type')
overlap[['osm_linkid',f'{other_name}_osm_type','coa_name','name','name_check','hausdorff_dist','geometry']].explore(color='red',m=m)#.sort_values('hausdorff_dist')

## Clean Up

In [ ]:
needed_cols = [f'{other_name}_id', f'{other_name}_osm_type', 'osm_linkid', 'geometry']

In [ ]:
#replace
overlap[[f'{other_name}_osm_type','facility_fwd']].value_counts(dropna=False)

In [ ]:
overlap.loc[(overlap[f'{other_name}_osm_type']=='cycletrack') & (overlap['facility_fwd']=='bike lane'),needed_cols+['facility_fwd']].explore(m=m)

In [ ]:
overlap = overlap[needed_cols]
overlap.rename(columns={f'{other_name}_osm_type':'improvement'},inplace=True)

In [ ]:
overlap.to_file(config['bicycle_facilities_fp']/"network_improvements.gpkg",layer='coa')

# Savannah Study Area 

In [ ]:
buffer_ft = 100
max_hausdorff_dist = 500
other_name = 'savannah'

In [ ]:
# import improvements
improvements = gpd.read_file(config['bicycle_facilities_fp']/"network_modifications.gpkg",layer='savannah')

In [ ]:
# copy to prevent modification
links_buffered = links.copy()
other_source = improvements.copy()

# buffer the osm cycleways
links_buffered.geometry = links_buffered.buffer(buffer_ft)

# intersect with coa/arc (returns coa/arc linestrings)
overlap = gpd.overlay(other_source,links_buffered)

#street name check if for bike lanes / sharrows / cycletracks
overlap['name'] = overlap['name'].apply(lambda row: matching_script.remove_suffix(row))
overlap[f"{other_name}_name"] = overlap[f"{other_name}_name"].apply(lambda row: matching_script.remove_suffix(row))
overlap['name_check'] = overlap.apply(lambda row: matching_script.name_check(row['name'],row[f"{other_name}_name"]),axis=1)

In [ ]:
# only accept matching names
overlap = overlap[(overlap['name_check']==True)]

In [ ]:
overlap.drop_duplicates(inplace=True)

In [ ]:
### HAUSDORFF DISTANCE CHECK ###
# add osm geometry to compare against arc/coa geometry
overlap = pd.merge(overlap,links[['osm_linkid','geometry']],on='osm_linkid')
overlap['hausdorff_dist'] = overlap.apply(lambda row: row['geometry_x'].hausdorff_distance(row['geometry_y']),axis=1)
overlap.drop(columns=['geometry_x'],inplace=True)
overlap.rename(columns={'geometry_y':'geometry'},inplace=True)

# replace intersected geometry with the original geometry
overlap = gpd.GeoDataFrame(overlap,geometry='geometry')

In [ ]:
# for remaining multi matches choose match with the lowest hausdorff distance
min_hausdorff = overlap.groupby('osm_linkid')['hausdorff_dist'].idxmin()
overlap = overlap.loc[min_hausdorff]

## Visually, it looks like most of the improvements were properly assigned

In [ ]:
improvements_buffer = improvements.copy()
improvements_buffer.geometry = improvements_buffer.buffer(200)
m = improvements_buffer.explore(f'{other_name}_osm_type')
overlap[[f'osm_linkid',f'{other_name}_osm_type',f'{other_name}_name','name','name_check','hausdorff_dist','geometry']].explore(color='red',m=m)#.sort_values('hausdorff_dist')

## Clean Up

In [ ]:
needed_cols = [f'{other_name}_id', f'{other_name}_osm_type', 'osm_linkid', 'geometry']

In [ ]:
# #replace
# overlap[[f'{other_name}_osm_type','facility_fwd']].value_counts(dropna=False)

In [ ]:
# overlap.loc[(overlap[f'{other_name}_osm_type']=='cycletrack') & (overlap['facility_fwd']=='bike lane'),needed_cols+['facility_fwd']].explore(m=m)

In [16]:
overlap = overlap[needed_cols]
overlap.rename(columns={f'{other_name}_osm_type':'improvement'},inplace=True)

In [17]:
overlap.to_file(config['bicycle_facilities_fp']/"network_improvements.gpkg",layer=f'{other_name}')